# Installations, imports, utils

In [ ]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

In [ ]:
pip install --upgrade langchain-community langchain-core


  Using cached langchain_community-0.2.7-py3-none-any.whl.metadata (2.5 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached marshmallow-3.21.3-py3-none-any.whl.metadata (7.1 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
Using cached langchain_community-0.2.7-py3-none-any.whl (2.2 MB)
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached marshmallow-3.21.3-py3-none-any.whl (49 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)


In [1]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [2]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma


d:\Minor Project\Exam preparator\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

# Load environment variables from .env file
load_dotenv()

# Retrieve the Hugging Face token from the environment variable
token = os.getenv("HUGGINGFACE_TOKEN")

if token:
    # Log in to Hugging Face
    login(token=token)
    print("Successfully logged in to Hugging Face!")
else:
    print("Hugging Face token not found. Please set it in the .env file.")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.


ValueError: Invalid token passed!

# Initialize model, tokenizer, query pipeline

Define the model, the device, and the `bitsandbytes` configuration.

In [9]:
model_id = 'mistralai/Mistral-7B-Instruct-v0.2'

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True
)

In [1]:
from langchain_community.llms import CTransformers
llm=CTransformers(model='SweatyCrayfish/llama-3-8b-quantized',
                      model_type='llama',
                      config={'max_new_tokens':256,
                              'temperature':0.01})

d:\Minor Project\Exam preparator\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]d:\Minor Project\Exam preparator\.venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\apoor\.cache\huggingface\hub\models--SweatyCrayfish--llama-3-8b-quantized. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Dev

ValidationError: 1 validation error for CTransformers
__root__
  No model file found in repo 'SweatyCrayfish/llama-3-8b-quantized' (type=value_error)

Prepare the model and the tokenizer.

In [10]:

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    
    device_map='auto',
)


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [3]:
import torch
print(f"Is CUDA available: {torch.cuda.is_available()}")

Is CUDA available: True


In [18]:
torch.cuda.empty_cache()

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Define the query pipeline.

In [35]:
time_1 = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
#         torch_dtype=torch.float16,
        device_map="auto",
        max_new_tokens=300)
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

Prepare pipeline: 0.0 sec.


We define a function for testing the pipeline.

In [ ]:
def test_model(tokenizer, pipeline, prompt_to_test):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        prompt_to_test: the prompt
    Returns
        None
    """
    # adapted from https://huggingface.co/blog/llama2#using-transformers
    time_1 = time()
    sequences = pipeline(
        prompt_to_test,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=len(prompt_to_test) + 256)
    time_2 = time()
    print(f"Test inference: {round(time_2-time_1, 3)} sec.")
#     for seq in sequences:
#         print(f"Result: {seq['generated_text']}")
    return sequences[0]['generated_text']
    

## Test the query pipeline

We test the pipeline with a query about the meaning of State of the Union (SOTU).

In [ ]:
test_model(tokenizer,
           query_pipeline,
           "Generate counterspeech for given hatespeech: Black people should be thrown out. Counterspeech: ?")

# Retrieval Augmented Generation

## Ingestion of Subject Materials

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-mpnet-base-v2"
# model_name = '/kaggle/input/stance-detect/transformers/1/1'
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

d:\Minor Project\Exam preparator\.conda\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
d:\Minor Project\Exam preparator\.conda\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\apoor\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment var

In [6]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
documents = []
path = "D:\Minor Project\Exam preparator\cloud_materials\cloud material"
for file in os.listdir(path):
    # print(file)
    file_path = path + "/" + file
    loader = PyPDFLoader(file_path)
    documents.extend(loader.load())
    text_splitter = CharacterTextSplitter(
        chunk_size= 800,
        chunk_overlap = 100,
        separator='', strip_whitespace=False
    )
    all_splits = text_splitter.split_documents(documents)
    vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")

In [15]:
from langchain_community.chat_models import ChatOllama
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser

model = ChatOllama(model="gemma:2b")
prompt = PromptTemplate.from_template(
            """
            <s> [INST] You are an expert in teaching.You will answer my queries in simple language using the context provided. Please 
                do not assume anything not provided in context.  [/INST] </s> 
            [INST] Question: {question} 
            Context: {context} 
            Answer: [/INST]
            """
        )
retriever = vectordb.as_retriever(
            search_type="similarity_score_threshold",
            search_kwargs={
                "k": 3,
                "score_threshold": 0.5,
            },
        )

chain = ({"context": retriever, "question": RunnablePassthrough()}
                | prompt
                | model
                | StrOutputParser())
query = "What is Cloud Native?"
chain.invoke(query)

"Sure, here's an answer to your question:\n\nThe context describes the **Cloud Native Stack**, which is a set of layers that developers use to build, manage, and run cloud-native applications. \n\nHere's a breakdown of the different layers:\n\n**Infrastructure Layer:**\n* This layer manages the underlying infrastructure required for cloud-native applications, including operating systems, storage, and networking.\n* Third-party cloud providers handle the management and maintenance of this infrastructure.\n\n**Provisioning Layer:**\n* This layer provides cloud services that allocate and configure the necessary resources for your application, such as virtual machines, storage, and databases.\n* It integrates with the infrastructure layer to provision the environment for your application.\n\n**Runtime Layer:**\n* This layer provides cloud-native technologies for running and managing your containerized application.\n* This includes services like cloud data storage, networking, and a contain

## Check the model with a HuggingFace pipeline


We check the model with a HF pipeline, using a query about the meaning of State of the Union (SOTU).

In [36]:
llm = HuggingFacePipeline(pipeline=query_pipeline)
# checking again that everything is working fine
llm(prompt="What is Cloud Native? ")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'What is Cloud Native?  Question: What is the difference between Cloud Native and Monolithic Architecture?\n\nAnswer: Cloud Native is a modern approach to building and deploying applications that utilizes containerization, microservices, and orchestration tools to enable continuous delivery, scalability, and resilience. It is designed to take full advantage of the benefits of the cloud computing environment.\n\nMonolithic architecture, on the other hand, is a traditional approach to building applications where all the components are tightly coupled and run as a single executable. Monolithic applications are deployed as a single unit and can be difficult to scale, update, and maintain.\n\nThe main differences between Cloud Native and Monolithic Architecture are:\n\n1. Scalability: Cloud Native applications can be scaled horizontally by adding more containers or nodes, while Monolithic applications require vertical scaling by adding more resources to a single server.\n2. Flexibility: Clo

## Creating Embeddings and Storing in Vector Store

Create the embeddings using Sentence Transformer and HuggingFace embeddings.

In [22]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 1.7 MB/s eta 0:00:00a 0:00:01m


In [23]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Initialize ChromaDB with the document splits, the embeddings defined previously and with the option to persist it locally.

In [25]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 966.8 kB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 4.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 6.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 4.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 5.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 5.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━

## Initialize chain

In [47]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

## Test the Retrieval-Augmented Generation 


We define a test function, that will run the query and time it.

In [48]:
def test_rag(qa, query):
    print(f"Query: {query}\n")
    time_1 = time()
    result = qa.run(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print("\nResult: ", result)

Let's check few queries.

In [50]:
query = "What is a Cloud Native application?"
test_rag(qa, query)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Query: What is a Cloud Native application?



> Entering new RetrievalQA chain...

> Finished chain.
Inference time: 15.282 sec.

Result:  Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Cloud Native Application
Cloud-native applications are software programs that consist of multiple 
small, interdependent services called microservices. 
•Traditionally, developers built monolithic applications with a single block 
structure containing all the required functionalities. 
•By using the cloud-native approach, software developers break the functionalities 
into smaller microservices. 
•This makes cloud-native applications more agile as these microservices work 
independently and take minimal computing resources to run.24

Cloud Native Application
Cloud-native applications are software programs that consist of multiple 
small, interdependent services called microservices. 
•T

In [ ]:
query = "What is the nation economic status? Summarize. Keep it under 200 words."
test_rag(qa, query)